In [2]:
from predictor.predict import predict

predict(
    smiles="COc1c(O)cc2oc3cc(O)c(CC=C(C)C)c(O)c3c(=O)c2c1CC=C(C)C", 
    model_name="catboost" # model name, look for available models in resources (model_name.pkl)
) # return pIC50

-4.152364264118942